Finding the Probability P(Y==1|X)

 Implementing Decision Function of SVM RBF Kernel.
 
 RBF kernel is defined as: $ K(x_{i},x_{q}) = exp(-\gamma ||x_{i} - x_{q}||^2) $



In [2]:
#Importing the packages.
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import numpy as np
from sklearn.svm import SVC

In [3]:
#making dataset X,y
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

In [4]:
#splitting the data into train,test,cv sets.
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
x_train,x_cv,y_train,y_cv= train_test_split(X_train,Y_train,test_size=0.25,random_state=1)

In [5]:
#Intializing the model SVC.
from sklearn.svm import SVC
clf=SVC(gamma=0.001,C=100)
clf.fit(x_train,y_train)
interc=clf.intercept_.shape

In [10]:
#Getting the values of support vevtors , n support, dual coefficents, intercept  values.
sv = clf.support_vectors_
nv = clf.n_support_
a  = clf.dual_coef_
b  = clf.intercept_
#This function c_kernel computes 𝐾(𝑥𝑖,𝑥𝑞)  = 𝑒𝑥𝑝(−𝛾||𝑥𝑖−𝑥𝑞||2)
def c_kernel(C,sv,X):
   ck =[(np.exp(-(C) * (X - ele)**2)) for ele in sv]
   ck1=[np.dot(ck,ele2[0]) for ele2 in clf.dual_coef_]
   return ck1
#Intializing the params.
params = 100
k = c_kernel(params, sv, X)
#This function decision_funtion (∑(𝑦𝑖𝛼𝑖𝐾(𝑥𝑖,𝑥𝑞))+𝑖𝑛𝑡𝑒𝑟𝑐𝑒𝑝𝑡)
def cdecision_funtion():
  f=c_kernel(params, sv, X)
  f1 = sum(f)+clf.intercept_
  return f1[10]
print("decision function from  custom implementation: ")
print(cdecision_funtion())

decision function from sklearn custom implemention: 
[[ -2.10562289  -2.10546417  -2.10546417  -2.10546417  -2.10546417]
 [ -2.10546418  -2.10546417 -25.82034402  -2.16761231  -2.10546417]
 [ -2.10546417  -2.10558024  -2.10546604  -2.10546417  -2.10546417]
 ...
 [ -2.10546417  -2.10546417 -16.95669346  -2.33733099  -2.10546417]
 [ -2.10546417  -2.10546417  -2.10546585  -2.10546417  -2.10546418]
 [ -2.10546417  -2.10546417 -83.83999001 -14.91235345  -2.10546417]]


In [11]:
# decision function from sklearn SVC implementaion.
print("deccision function from sklearn implementation")
print(clf.decision_function(x_cv))

deccision function from sklearn implementation
[-3.04107788e+00 -1.41113141e+00 -2.23654838e+00  1.94133676e+00
  1.60471505e+00 -2.49670518e+00 -1.48081364e+00 -3.44733079e+00
  1.43827857e+00  1.51389734e+00 -3.54685914e+00 -1.79434464e+00
  2.04411153e+00  6.28557197e-01 -2.50214127e+00 -2.43429841e+00
 -3.54806396e-01 -4.58355870e+00  1.14332107e+00 -2.38898263e+00
 -1.22825736e+00  2.50236644e+00  3.01740061e+00 -2.74920121e+00
 -2.41784620e+00 -3.47707090e+00 -1.75412320e+00 -1.39850718e+00
 -2.28433240e+00  1.79881585e+00 -5.84209777e-01  1.85468425e+00
 -3.32836991e+00 -3.82687851e+00  1.30222111e+00 -2.98881968e+00
 -2.15818885e+00 -4.05877306e+00  3.08318933e-01 -3.38278372e+00
 -1.14147129e-02  1.96947177e+00  2.05232281e+00 -3.95278664e-01
 -2.99772936e+00  2.12695915e+00 -2.36223295e+00 -4.29289667e+00
  1.23276331e+00 -3.87716441e+00 -1.73239754e+00 -2.05400117e+00
 -7.88371337e-01  1.56789113e+00 -1.93643623e+00  7.78484280e-01
 -4.73156306e+00 -2.70188589e+00  2.1766341

Implementing Platt Scaling to find P(Y==1|X)

Check this <a href='https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a'>PDF</a>
<img src='https://i.imgur.com/CAMnVnh.png'>



>  Applying SGD algorithm with ($f_{cv}$, $y_{cv}$) and finding the weights $W$ intercept $b$ . Here our data is of one dimensional so we will have a one dimensional weight vector i.e W.shape (1,)

> We have to  change the values of $y_{cv}$ as mentioned in the above image. you will calculate y+, y- based on data points in train data

> The Sklearn's SGD algorithm doesn't support the real valued outputs
<img src='https://i.imgur.com/zKYE9Oc.png'>
if Y[i] is 1, it will be replaced with y+ value else it will replaced with y- value

>  For a given data point from $X_{test}$, $P(Y=1|X) = \frac{1}{1+exp(-(W*f_{test}+ b))}$ where ` `$f_{test}$ ```= decision_function(```$X_{test}$```)```, W and b will be learned as metioned in the above step

In [12]:
#Modifying y_cv according to given instructions and storing in f_p and f_n 
f_p1=(y_cv==0).sum(axis=0)
f_p=(f_p1+1)/(f_p1+2)
f_n1=(y_cv==1).sum(axis=0)
f_n=1/(f_n1+2)
f_p,f_n

(0.9986033519553073, 0.003472222222222222)

In [13]:
#This sig function applies sigmoid .
def sig(w,x,b):
  return  1/(1+np.exp(-(np.dot((w.T),x)+b)))
#This logloss function( modified) calculates the loss loss.
def logloss(w,b,x,y):
  n=len(x)
  loss=0
  for i in range(n):
    loss+=f_p*np.log10(sig(w,x[i],b))+f_n*np.log10(1-sig(w,x[i],b))
  loss= -1 * (loss/n)
  return loss

In [18]:
#This function intialize_weights creates a weights of zers and returns the weights and bias.
def initialize_weights(dim):
  x1=np.arange(len(dim))
  x2=x1.reshape((len(dim)))
  w=np.zeros_like(x2)
  bias=int(False)
  return w,bias
#This function gradient_dw use to update weights.
def gradient_dw(x,y,w,b,alpha,N):
  z=np.dot((w.T),x)+b   
  dw=x*(y-sigmoid(z))-(alpha*w)/N 
  return dw
#This function gradient_db use to update bias.
def gradient_db(x,y,w,b): 
  z=np.dot((w.T),x)+b 
  db=y-sigmoid(z)
  return db
def sigmoid(z): 
  s=1/(1+np.exp(-z)) 
  return s

In [15]:
#This function train is getting from previous assignment it returns the weights and bias. 
def train(x_train,y_train,epochs,alpha,eta0):
  N=len(x_train)
  train_log_loss=[]
  test_log_loss=[]
  wi=[]
  bi=[] 
  w,b=initialize_weights(x_train[0]) 
  for i in range(1,epochs+1):
    print("epoch : %s"%i) 
    for ele in range(N):
      gdw=gradient_dw(x_train[ele],y_train[ele],w,b,alpha,N) 
      gdb=gradient_db(x_train[ele],y_train[ele],w,b) 
      w=np.array(w)+(eta0*(np.array(gdw))) 
      b=b+(alpha*(gdb))
  return w,b

In [16]:
%time
#Intializing the parameters.
alpha=0.0001
eta0=0.0001
N=len(x_train)
epochs=15 

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


In [19]:
#Training the model and getting weights and bias
weights,binter=train(x_cv,y_cv,epochs,alpha,eta0)

epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9
epoch : 10
epoch : 11
epoch : 12
epoch : 13
epoch : 14
epoch : 15


In [26]:
#This function prob returns the custom implementation of decision function.
def prob():
  cdf=cdecision_funtion()
  pp=[]
  for ele2 in x_test:
    p=1/(1+np.exp(-np.dot(weights,cdf.T)+binter))
  pp.append(p)
  return pp
po=prob()
print("for each point in x_test ,𝑃(𝑌=1|𝑋) = ")
print(po)

for each point in x_test ,𝑃(𝑌=1|𝑋) = 
[array([0.52101451, 0.97277586, 0.5210155 , ..., 0.90942512, 0.52101472,
       0.99999947])]
